<a href="https://colab.research.google.com/github/crisprking/miniprotein_genai/blob/main/notebooks/01_backbone_RFdiffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔧 Install RFdiffusion & helpers (≈3 min on Colab GPU)
%%bash
set -e

# 1.  Basic Python deps
python -m pip install --quiet --upgrade pip
python -m pip install --quiet py3Dmol omegaconf hydra-core jedi \
    git+https://github.com/NVIDIA/dllogger#egg=dllogger

# 2.  DGL + e3nn (GPU wheels, no heavy CUDA deps pulled)
python -m pip install --quiet --no-dependencies \
    dgl==2.0.0 -f https://data.dgl.ai/wheels/cu121/repo.html
python -m pip install --quiet --no-dependencies e3nn==0.5.5 opt_einsum_fx

# 3.  Clone RFdiffusion (if not already)
if [ ! -d RFdiffusion ]; then
  git clone --depth 1 https://github.com/sokrypton/RFdiffusion.git
fi

# 4.  Build & install the SE3Transformer sub‑module
python -m pip install --quiet ./RFdiffusion/env/SE3Transformer

# 5.  ColabDesign (this repo *is* pip‑packaged)
python -m pip install --quiet git+https://github.com/sokrypton/ColabDesign.git

echo "✅  RFdiffusion + ColabDesign environment ready"


In [ ]:
# ⚙️ User parameters
name        = "design1"   #@param {type:"string"}
length      = 100         #@param {type:"integer"}
symmetry    = "none"      #@param ["none","cyclic","dihedral"]
order       = 1           #@param {type:"integer"}
num_designs = 1           #@param {type:"integer"}
iterations  = 50          #@param ["25","50","100","150","200"] {type:"raw"}
print(f"Backbone length {length}, copies {num_designs}")


In [ ]:
# 🚀 Run RFdiffusion
import subprocess
contigs = f"{length}"
out_prefix = f"outputs/{name}"

cmd = [
    "python","-m","RFdiffusion.run_inference",
    f"contigmap.contigs=[{contigs}]",
    f"inference.output_prefix={out_prefix}",
    f"inference.num_designs={num_designs}",
    f"diffuser.T={iterations}",
    "inference.dump_pdb=True",
    "inference.dump_pdb_path=/dev/shm"
]
if symmetry != "none":
    symflag = f"{symmetry[0]}{order}"
    cmd = ["python","-m","RFdiffusion.run_inference","--config-name","symmetry",
           f"inference.symmetry={symflag}"] + cmd[2:]

print("Running:\n", " ".join(cmd))
subprocess.check_call(cmd)
print("✅ Backbone(s) finished; files in /content/outputs/")


In [ ]:
# 👀 Preview first backbone
import glob, py3Dmol
pdb = glob.glob(f"outputs/{name}_0.pdb")[0]
view = py3Dmol.view(js='https://3dmol.org/build/3Dmol.js')
view.addModel(open(pdb).read(),'pdb')
view.setStyle({'cartoon':{'color':'spectrum'}})
view.zoomTo(); view.show()


In [ ]:
# 📦 Download result
import shutil, google.colab
zip_name = f"{name}.result"
shutil.make_archive(zip_name, "zip", root_dir="outputs", base_dir=name)
google.colab.files.download(f"{zip_name}.zip")
